<a href="https://colab.research.google.com/github/1rd0/apple_vision_pro_for_cheap1/blob/main/%D0%9F%D1%80%D0%B5%D0%B4%D0%B8%D0%BA%D1%82_%D1%82%D1%80%D0%B5%D1%82%D0%B8%D1%87%D0%BD%D0%BE%D0%B9_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B1%D0%B5%D0%BB%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython pandas numpy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.4 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
from Bio.PDB import PDBList

# Скачивание PDB файла
pdb_id = '1TUP'  # Пример PDB ID
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, file_format='pdb')


'/content/tu/pdb1tup.ent'

In [4]:
import numpy as np
from Bio.PDB import PDBParser
from Bio import BiopythonWarning
import numpy as np
from Bio.PDB import PDBParser, PDBList
import gzip
import os
import warnings


warnings.simplefilter('ignore', BiopythonWarning)

 # Извлечение аминокислотных последовательностей
def extract_sequence(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_file)
    sequences = []

    for model in structure:
        for chain in model:
            sequence = ''.join([residue.get_resname() for residue in chain if residue.get_id()[0] == ' '])
            sequences.append(sequence)

    return sequences

sequences = extract_sequence(f'pdb1tup.ent')
print(sequences)

FileNotFoundError: [Errno 2] No such file or directory: 'pdb1tup.ent'

In [6]:
import numpy as np
from Bio.PDB import PDBParser
from Bio import BiopythonWarning
import numpy as np
from Bio.PDB import PDBParser, PDBList
import gzip
import os
import warnings


warnings.simplefilter('ignore', BiopythonWarning)


# Извлечение аминокислотных последовательностей
def extract_sequence(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_file)
    sequences = []

    for model in structure:
        for chain in model:
            sequence = ''.join([residue.get_resname() for residue in chain if residue.get_id()[0] == ' '])
            sequences.append(sequence)

    return sequences

sequences = extract_sequence(f'pdb1tup.ent')

# Преобразование последовательностей в one-hot encoding
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: idx for idx, aa in enumerate(amino_acids)}

def one_hot_encode_sequence(sequence):
    encoding = np.zeros((len(sequence), len(amino_acids)), dtype=int)
    for i, aa in enumerate(sequence):
        if aa in aa_to_idx:
            encoding[i, aa_to_idx[aa]] = 1
    return encoding

encoded_sequences = [one_hot_encode_sequence(seq) for seq in sequences]

# Извлечение координат атомов
def extract_coordinates(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_file)
    coordinates = []

    for model in structure:
        for chain in model:
            chain_coords = []
            for residue in chain:
                if 'CA' in residue:  # Используем только C-альфа атомы
                    atom = residue['CA']
                    chain_coords.append(atom.coord)
            coordinates.append(chain_coords)

    return coordinates

coordinates = extract_coordinates(f'pdb1tup.ent')

# Преобразование координат в numpy массив
encoded_coordinates = [np.array(coords) for coords in coordinates]

# Разделение данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = np.array(encoded_sequences, dtype=object)
y = np.array(encoded_coordinates, dtype=object)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Сохранение данных в файлы
import pickle

with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open('y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)


In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle

# Функция для паддинга последовательностей до одинаковой длины
def pad_sequences(sequences, max_len, pad_value=0):
    padded_sequences = np.full((len(sequences), max_len, sequences[0].shape[1]), pad_value)
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = seq
    return padded_sequences

# Функция для паддинга координат до одинаковой длины
def pad_coordinates(coordinates, max_len, pad_value=0.0):
    padded_coordinates = np.full((len(coordinates), max_len, 3), pad_value)
    for i, coord in enumerate(coordinates):
        if len(coord) > 0:
            padded_coordinates[i, :len(coord)] = coord
    return padded_coordinates


# Загрузка данных
with open('X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open('y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

# Найдем максимальную длину последовательности и координат
max_seq_len = max(max(len(seq) for seq in X_train), max(len(seq) for seq in X_test))
max_coord_len = max(max(len(coord) for coord in y_train), max(len(coord) for coord in y_test))

# Паддинг последовательностей
X_train_padded = pad_sequences(X_train, max_seq_len)
X_test_padded = pad_sequences(X_test, max_seq_len)

# Паддинг координат
y_train_padded = pad_coordinates(y_train, max_coord_len)
y_test_padded = pad_coordinates(y_test, max_coord_len)
 # Преобразование данных в формат PyTorch
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_padded.reshape(len(y_train_padded), -1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_padded.reshape(len(y_test_padded), -1), dtype=torch.float32)


# Создание DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Класс модели RNN
class ProteinRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(ProteinRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # берем последнее временное значение
        return out

# Гиперпараметры
input_size = len(amino_acids)  # размер one-hot encoding для аминокислот
hidden_size = 128
output_size = y_train_tensor.shape[1]  # количество координат (3 * число атомов)
num_layers = 2
num_epochs = 3
learning_rate = 0.001

# Инициализация модели, функции потерь и оптимизатора
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ProteinRNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        # Перемещение данных на устройство (CPU/GPU)
        inputs, targets = inputs.to(device), targets.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Вычисление функции потерь
        loss = criterion(outputs, targets)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Сохранение обученной модели
torch.save(model.state_dict(), 'protein_rnn_model.pth')

# Функция для оценки модели
def evaluate_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)

# Оценка модели
test_loss = evaluate_model(model, test_loader)
print(f'Test Loss: {test_loss:.4f}')


Epoch [1/3], Loss: 2664.6003
Epoch [2/3], Loss: 2662.7729
Epoch [3/3], Loss: 2660.1714
Test Loss: 0.0403


In [16]:
print(X_train)


[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]) array([[0, 0, 0, ..., 0, 0, 0],
                                         [0, 0, 0, ..., 0, 0, 0],
                                         [0, 0, 0, ..., 0, 0, 0],
                                         ...,
                                         [0, 0, 0, ..., 0, 0, 0],
                                         [0, 0, 0, ..., 0, 0, 0],
                                         [0, 0, 0, ..., 0, 0, 0]])
 array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0,